# Importing Libraries

In [59]:
import matplotlib.pyplot as plt
import numpy as np

from DataSet import DataSet as DS

%matplotlib qt

# Building A Mesh and Loading $\vec{B}$
I used one of Daniel's data (output of MFM simulation)

In [60]:
BaseAddress = "/home/farhadda/BackUp/"

file = BaseAddress+'div_E_omega_2017-09-06T11-24-00.npz'
data = np.load(file, allow_pickle=True, encoding="bytes")
ds1 = DS(SystemOfCoords="CAR", BaseAddress=BaseAddress,
         Pattern="div_E_omega_2017-09-06T11-24-00.npz",
         NBlocks=1, UseBlock=0)
Beg = ds1.geo['num_ghost_cells'][0]
for v in ds1.geo["vars"].keys():
    ds1.Scalar(v, ds1.geo["vars"][v]["Location"], ds1.Adummy)
    if Beg > 0:
        ds1.vars[v]["val"] = data[v][Beg:-Beg, Beg:-Beg, Beg:-Beg]
    else:
        ds1.vars[v]["val"] = data[v]

---<( Input files include Ghost-Cells, those cells will be ignored )>---
Following parameters are found in the given file :: 
 Bx (defined on FaceX)
 By (defined on FaceY)
 Bz (defined on FaceZ)


# Check Div. B for input Magnetic Field

In [61]:
ds1.DivFace("Bx", "By", "Bz", "DivB")
a = np.abs(ds1.vars["DivB"]['val'])
print(a.mean(), a.max(), a.min(), sep='\t')
print(*np.where(a == a.max()))

4.460892352578051e-05	0.2916412353515625	0.0
[108] [98] [0]


# Notice :
It seems that input magnetic field is not  Div. Free, so to test "Moving to new mesh" method I will use the magnetic field extracted from vector potential field which itself is extracted from input magnetic field data

Due to previous problems in extraction of $\vec{A}$ from $\vec{B}$ I need to implement this method differently:
\begin{eqnarray}
\beta_{x}(x, y_{1}) &=& 0 \\
\beta_{y}(x_{1}, y) &=& 0
\end{eqnarray}

\begin{eqnarray}
\beta_x(x, y) &=& - \frac{1}{2} \int_{y_1}^{y} dy' B_z(x, y', z_2) \\
\beta_y(x, y) &=&   \frac{1}{2} \int_{x_1}^{x} dx' B_z(x', y, z_2)
\end{eqnarray}


In [66]:
dx = ds1.nodevect['X'][1:] - ds1.nodevect['X'][:-1]
dy = ds1.nodevect['Y'][1:] - ds1.nodevect['Y'][:-1]
dz = ds1.nodevect['Z'][1:] - ds1.nodevect['Z'][:-1]
BetaX = np.empty_like(ds1.EdgeX["X"][:, :, 0])
BetaY = np.empty_like(ds1.EdgeY["Y"][:, :, 0])

BetaX[:, 0] = 0.0
BetaY[0, :] = 0.0
BzOnTop = ds1.vars["Bz"]['val'][:, :, -1]
for j in range(1, dy.size + 1):
    for i in range(dx.size):
        BetaX[i, j] = -0.5 * np.sum(dy[:j] * BzOnTop[i, :j])

for i in range(1, dx.size + 1):
    for j in range(dy.size):
        BetaY[i, j] = 0.5 * np.sum(dx[:i] * BzOnTop[:i, j])

\begin{eqnarray}
A_x(x, y, z) &=& \beta_x(x, y) - \int_{z}^{z_2} dz' B_y(x, y, z') \\
A_y(x, y, z) &=& \beta_y(x, y) + \int_{z}^{z_2} dz' B_x(x, y, z') \\
A_z(x, y, z) &=& 0
\end{eqnarray}

In [67]:
ds1.Scalar("Ax", "EdgeX", ds1.Adummy)
ds1.Scalar("Ay", "EdgeY", ds1.Adummy)
ds1.Scalar("Az", "EdgeZ", ds1.Adummy)

In [68]:
ds1.vars["Ax"]['val'][:, :, -1] = BetaX[:, :]
ds1.vars["Ay"]['val'][:, :, -1] = BetaY[:, :]

In [75]:
for k in range(-2, -dz.size-2, -1):
    BetaX[:, :] = BetaX[:, :] - dz[k + 1] * ds1.vars["By"]['val'][:, :, k + 1]
    BetaY[:, :] = BetaY[:, :] + dz[k + 1] * ds1.vars["Bx"]['val'][:, :, k + 1]
    ds1.vars["Ax"]['val'][:, :, k] = BetaX[:, :]
    ds1.vars["Ay"]['val'][:, :, k] = BetaY[:, :]

Now test if $\nabla \times \vec{A}$ can reproduce the initial magnetic field with Div. free condition

In [77]:
ds1.CurlEdgeToFace("Ax", "Ay", "Az", "ABx", "ABy", "ABz")
ds1.DivFace("ABx", "ABy", "ABz", "DivAB")

In [85]:
a = np.abs(ds1.vars["DivAB"]['val'])
print(a.mean(), a.max(), a.min(), sep='\t')
print(np.where(a == a.min())[0].shape)

2.0496285899200763e-07	0.0001220703125	0.0
(3465780,)


In [92]:
plt.figure()
plt.imshow((ds1.vars["Bx"]['val'][:, :, 0] - ds1.vars["ABx"]['val'][:, :, 0]).T, origin="Lower Left")
plt.tight_layout()
plt.title(r"Diff $B_{x}$ on Bottom")
plt.colorbar()

In [93]:
plt.figure()
plt.imshow((ds1.vars["By"]['val'][:, :, 0] - ds1.vars["ABy"]['val'][:, :, 0]).T, origin="Lower Left")
plt.tight_layout()
plt.title(r"Diff $B_{y}$ on Bottom")
plt.colorbar()

In [94]:
plt.figure()
plt.imshow((ds1.vars["Bz"]['val'][:, :, 0] - ds1.vars["ABz"]['val'][:, :, 0]).T, origin="Lower Left")
plt.tight_layout()
plt.title(r"Diff $B_{z}$ on Bottom")
plt.colorbar()